In [78]:
import asyncio
from aiohttp import ClientSession
import pandas as pd
from urllib.parse import urlparse

In [97]:
all_urls = pd.read_csv('urls.csv')['url'].values
all_urls_with_port = []

for url in all_urls:
    parsed = urlparse(url)
    if parsed.scheme == "https":
        replaced = parsed._replace(netloc=parsed.netloc + ":443")
    else:
        replaced = parsed._replace(netloc=parsed.netloc + ":80")
        
    all_urls_with_port.append(replaced.geturl())

In [102]:
async def fetch(url, session):
    async with session.head(url, allow_redirects=True) as response:
        return response.url

In [103]:
async def bound_fetch(sem, url, session):
    # Getter function with semaphore.
    async with sem:
        await fetch(url, session)

In [104]:
async def run(r):
    global all_urls
    tasks = []
    # create instance of Semaphore
    sem = asyncio.Semaphore(1000)

    # Create client session that will ensure we dont open new connection
    # per each request.
    async with ClientSession() as session:
        for i in range(r):
            # pass Semaphore and session to every GET request
            task = asyncio.ensure_future(bound_fetch(sem, all_urls_with_port[i], session))
            tasks.append(task)

        responses = asyncio.gather(*tasks)
        await responses

In [ ]:
number = 10000
loop = asyncio.get_event_loop()

future = asyncio.ensure_future(run(number))
loop.run_until_complete(future)